In [2]:
from __future__ import annotations

from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib.patches import Patch
from tqdm import tqdm

import gymnasium as gym


# Let's start by creating the blackjack environment.

env = gym.make('Blackjack-v1', natural=False, sab=False)
# Whether to follow the exact rules outlined in the book by Sutton and Barto. If `sab` is `True`, the keyword argument `natural` will be ignored.

We call env.reset() to start an episode. This function resets the environment to a starting position and returns an initial observation. We usually also set done = False. This variable will be useful later to check if a game is terminated (i.e., the player wins or loses).

In [5]:
# reset the environment to get the first observation
done = False
observation, info = env.reset()

# observation = (16, 9, False)
    # Note that our observation is a 3-tuple consisting of 3 values:
    # The players current sum
    # Value of the dealers face-up card
    # Boolean whether the player holds a usable ace (An ace is usable if it counts as 11 without busting)

env.step(action) function to interact with the environment. This function takes an action as input and executes it in the environment

In [4]:
# sample a random action from all valid actions
action = env.action_space.sample()
# action=1

# execute the action in our environment and receive infos from the environment
observation, reward, terminated, truncated, info = env.step(action)

# observation=(24, 10, False)
# reward=-1.0
# terminated=True
# truncated=False
# info={}